In [9]:
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
#from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import backend
import numpy as np
from tensorflow.keras.preprocessing import image

In [10]:
import sagemaker
sagemaker.get_execution_role()

'arn:aws:iam::474422712127:role/sagemaker-role-BYOC'

In [11]:
#model = tf.keras.applications.ResNet50()
tf.keras.backend.set_learning_phase(0)
model = resnet50.ResNet50()

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [12]:
export_dir = "00002"

In [13]:
tf.keras.backend.set_learning_phase(0)
model = tf.keras.applications.ResNet50()

if not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print("Directory ", export_dir, " Created ")
else:
    print("Directory ", export_dir, " already exists")
# Save to SavedModel
model.save(export_dir, save_format="tf", include_optimizer=False)

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-11-14 18:54:23.332271: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-14 18:54:23.332310: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-36-57.ec2.internal): /proc/driver/nvidia/version does not exist
2022-11-14 18:54:23.332739: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-criti

Directory  00002  Created 


2022-11-14 18:54:29.955972: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 00002/assets


/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [14]:
!tar -cvpzf model.tar.gz ./00002

./00002/
./00002/variables/
./00002/variables/variables.data-00000-of-00001
./00002/variables/variables.index
./00002/keras_metadata.pb
./00002/saved_model.pb
./00002/assets/


In [2]:
import sagemaker
sagemaker_session = sagemaker.Session()

In [3]:
role = sagemaker.get_execution_role()

In [4]:
model_url = sagemaker_session.upload_data(path="model.tar.gz", key_prefix="resnet-tf-repack")

In [5]:
model_url

's3://sagemaker-us-east-1-474422712127/resnet-tf-repack/model.tar.gz'

In [6]:
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess


#Setup
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
region = boto_session.region_name
print(region)

us-east-1


In [7]:
# retrieve tf image
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region="us-east-1",
    version="2.3.0",
    py_version="py3",
    instance_type="ml.m5.xlarge",
    image_scope="inference"
)

In [8]:
#Step 1: Model Creation
model_name = "tf-resnet" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)
create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "SingleModel",
            "ModelDataUrl": model_url,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': model_url} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: tf-resnet2022-11-14-18-56-10
Model Arn: arn:aws:sagemaker:us-east-1:474422712127:model/tf-resnet2022-11-14-18-56-10


In [9]:
#Step 2: EPC Creation
epc_name = "tf-resnet" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            "VariantName": "tfvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.xlarge",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint-config/tf-resnet2022-11-14-18-56-13


In [10]:
#Step 3: EP Creation
endpoint_name = "tf-resnet" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint/tf-resnet2022-11-14-18-56-16


In [11]:
#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'tf-resnet2022-11-14-18-56-16', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:474422712127:endpoint/tf-resnet2022-11-14-18-56-16', 'EndpointConfigName': 'tf-resnet2022-11-14-18-56-13', 'ProductionVariants': [{'VariantName': 'tfvariant', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu', 'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:91ebb7428846c5f7b515d5d9b8389a14c73d0c5d02657f4a6413592124333278', 'ResolutionTime': datetime.datetime(2022, 11, 14, 18, 56, 17, 497000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2022, 11, 14, 18, 56, 16, 858000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 11, 14, 18, 57, 54, 841000, tzinfo=tzlocal()), 

In [12]:
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras import backend
import numpy as np
from tensorflow.keras.preprocessing import image

In [13]:
# Load the image file, resizing it to 224x224 pixels (required by this model)
img = image.load_img("kitten.jpg", target_size=(224, 224))
# Convert the image to a numpy array
x = image.img_to_array(img)
# Add a forth dimension since Keras expects a list of images
x = np.expand_dims(x, axis=0)

# Scale the input image to the range used in the trained network
x = resnet50.preprocess_input(x)
print(type(x))

<class 'numpy.ndarray'>


In [14]:
payload = json.dumps(x.tolist())
#payload

In [15]:
endpoint_name

'tf-resnet2022-11-14-18-56-16'

In [17]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='application/json',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
result

{'predictions': [[9.12162701e-09,
   5.53907498e-08,
   2.70708211e-09,
   3.53683554e-08,
   9.56350554e-09,
   4.63316674e-08,
   2.23814354e-07,
   1.60062157e-07,
   1.22796439e-06,
   1.98623047e-06,
   4.99394491e-06,
   4.02949183e-07,
   3.62503016e-07,
   5.94971681e-08,
   1.31474692e-07,
   2.59633367e-07,
   4.07241068e-08,
   3.77016448e-07,
   2.20471165e-07,
   7.83861154e-09,
   5.99396458e-07,
   6.724988e-07,
   5.17729177e-07,
   5.65231289e-07,
   3.27209164e-05,
   4.06752783e-08,
   6.6239852e-07,
   6.47027e-07,
   1.13020135e-08,
   3.09540376e-08,
   1.39157819e-05,
   5.02169939e-07,
   3.1026741e-06,
   7.24140037e-08,
   2.39608084e-07,
   1.44620469e-06,
   5.57536964e-07,
   4.19473821e-07,
   2.76209562e-06,
   1.01493324e-05,
   2.35100288e-07,
   3.75449199e-05,
   1.34497634e-06,
   3.03819184e-06,
   5.27983548e-06,
   4.65989842e-07,
   2.39542146e-06,
   6.11868813e-07,
   1.0017834e-06,
   7.50386607e-06,
   7.90211686e-07,
   2.29059694e-07,
   4.